In [9]:
%pip install tensorflow
%pip install pillow


Note: you may need to restart the kernel to use updated packages.
  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl (2.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize input data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

# One-hot encode target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 12s 1us/step


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for digits 0-9
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


c:\Users\satya\OneDrive\Desktop\TASK2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.8681 - loss: 0.4689 - val_accuracy: 0.9850 - val_loss: 0.0496
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9824 - loss: 0.0550 - val_accuracy: 0.9871 - val_loss: 0.0390
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9884 - loss: 0.0381 - val_accuracy: 0.9889 - val_loss: 0.0310
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9908 - loss: 0.0278 - val_accuracy: 0.9907 - val_loss: 0.0306
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9932 - loss: 0.0212 - val_accuracy: 0.9908 - val_loss: 0.0301


In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")  # Should be over 98%


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9883 - loss: 0.0387
Test accuracy: 0.9908


In [11]:
model.save('mnist_model.h5')

In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from PIL import Image, ImageDraw, ImageTk
import tensorflow as tf
import os

class DigitRecognizerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("MNIST Digit Recognizer")
        
        # Check if model exists
        model_path = 'mnist_model.h5'
        if not os.path.exists(model_path):
            messagebox.showerror("Error", "Model file 'mnist_model.h5' not found!")
            self.destroy()
            return
        
        try:
            # Load model with error handling
            self.model = tf.keras.models.load_model(model_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model:\n{str(e)}")
            self.destroy()
            return
        
        # Create UI
        self.create_widgets()
        
    def create_widgets(self):
        """Create all UI components"""
        # Main frame
        main_frame = tk.Frame(self, padx=20, pady=20)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Drawing canvas
        self.canvas = tk.Canvas(main_frame, width=280, height=280, bg='black', 
                               highlightthickness=1, highlightbackground="gray")
        self.canvas.grid(row=0, column=0, columnspan=2, pady=10)
        
        # Button frame
        button_frame = tk.Frame(main_frame)
        button_frame.grid(row=1, column=0, columnspan=2, pady=10)
        
        # Buttons
        predict_btn = tk.Button(button_frame, text="Predict", width=10, 
                               command=self.predict)
        predict_btn.pack(side=tk.LEFT, padx=5)
        
        clear_btn = tk.Button(button_frame, text="Clear", width=10, 
                             command=self.clear_canvas)
        clear_btn.pack(side=tk.LEFT, padx=5)
        
        # Status label
        self.status = tk.Label(main_frame, text="Draw a digit in the black area", 
                              fg="blue", font=("Arial", 10))
        self.status.grid(row=2, column=0, columnspan=2, pady=5)
        
        # Initialize drawing tools
        self.image = Image.new("L", (280, 280), 0)  # Black background
        self.draw = ImageDraw.Draw(self.image)
        self.last_x, self.last_y = None, None
        
        # Bind mouse events
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset)
        
        # Test image display
        self.test_img = None
    
    def paint(self, event):
        """Draw on both tkinter canvas and PIL image"""
        if self.last_x and self.last_y:
            # Draw line between points
            self.canvas.create_line(self.last_x, self.last_y, event.x, event.y,
                                   fill='white', width=15, capstyle=tk.ROUND, 
                                   smooth=tk.TRUE)
            self.draw.line([self.last_x, self.last_y, event.x, event.y], 
                          fill=255, width=15)
        
        # Draw circle at current position
        radius = 7
        self.canvas.create_oval(event.x-radius, event.y-radius, 
                               event.x+radius, event.y+radius,
                               fill='white', outline='white')
        self.draw.ellipse([event.x-radius, event.y-radius, 
                          event.x+radius, event.y+radius], fill=255)
        
        self.last_x, self.last_y = event.x, event.y
        self.status.config(text="Drawing... Click 'Predict' when ready")

    def reset(self, event):
        """Reset drawing coordinates"""
        self.last_x, self.last_y = None, None

    def clear_canvas(self):
        """Clear the drawing area"""
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), 0)  # Reset to black
        self.draw = ImageDraw.Draw(self.image)
        self.status.config(text="Canvas cleared. Draw a new digit")

    def predict(self):
        """Predict the drawn digit"""
        try:
            # Preprocess image
            img = self.image.resize((28, 28)).convert('L')
            
            # Convert to numpy array and normalize
            img_array = np.array(img)
            if np.max(img_array) > 0:  # Check if something was drawn
                img_array = img_array.reshape(1, 28, 28, 1) / 255.0
                
                # Make prediction
                pred = self.model.predict(img_array)
                digit = np.argmax(pred)
                confidence = np.max(pred)
                
                # Show result
                messagebox.showinfo("Prediction", 
                                   f"Predicted Digit: {digit}\nConfidence: {confidence:.2%}")
                self.status.config(text=f"Predicted: {digit} (Confidence: {confidence:.2%})")
            else:
                messagebox.showwarning("Warning", "Please draw a digit first!")
                self.status.config(text="Please draw a digit first!")
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed:\n{str(e)}")
            self.status.config(text="Prediction error")

if __name__ == "__main__":
    app = DigitRecognizerApp()
    app.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
